In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# since I added the weights file of the pretrained Resnet50 model to this kernel, there're two folders in the ../input
# ../input/plant-seedlings-classification and ../input/ResNet-50
print(os.listdir("../input"), os.listdir("../input/plant-seedlings-classification/train"))
# Any results you write to the current directory are saved as output.

In [ ]:
# how many files in each category
training_base_dir = "../input/plant-seedlings-classification/train"
file_nums = []
category_files = {}
for directory in os.listdir(training_base_dir):
    category_files[directory] = []
    cate_dir = os.path.join(training_base_dir, directory)
    file_num = 0
    for file_name in os.listdir(cate_dir):
        full_file_name = os.path.join(cate_dir, file_name)
        category_files[directory].append(full_file_name)
        file_num += 1
    print(cate_dir, file_num)
    file_nums.append(file_num)
        

In [ ]:
# create folders for training and validation data
base_dir = "./"
categories = os.listdir("../input/plant-seedlings-classification/train")
# I initially wanted to create 4 training datasets and 4 validation datasets and use theses datasets to train different models.
# Since there's only 5 GB disk I can use, only one training dataset and one validation dataset are created.
datasets_num = 1
for idx in range(1, datasets_num + 1):
    train_val_str = [data_type + str(idx) for data_type in ["train", "val"]]
    for data_type in train_val_str:
        tmp_path0 = os.path.join(base_dir, data_type)
        try:
            os.mkdir(tmp_path0)
        except (FileExistsError, FileNotFoundError):
            print("raise an error when creating {}".format(tmp_path))
            continue        
        for category in categories:
            tmp_path1 = os.path.join(tmp_path0, category)
            try:
                os.mkdir(tmp_path1)
            except (FileExistsError, FileNotFoundError):
                print("raise an error when creating {}".format(tmp_path))
                continue

In [ ]:
# sample files and copy these files to training and validation dataset folders
from shutil import copy
from random import sample, seed
from pdb import set_trace
seed()
for i in range(1, datasets_num + 1):
    for _, category in enumerate(category_files.keys()):
        l = len(category_files[category])
        train_data_num = int(l * 0.9)
        valid_data_num = l - train_data_num
        files2copy = sample(category_files[category], l)
        train_dest = os.path.join(base_dir, "train{}".format(i), category)
        valid_dest = os.path.join(base_dir, "val{}".format(i), category)
        for j in range(train_data_num):
            copy(files2copy[j], train_dest)
        for j in range(train_data_num, l):
            copy(files2copy[j], valid_dest)
            

In [ ]:
# create a folder to store the weights of pretrained models
from os.path import join, exists
from os import makedirs
cache_dir = join('/tmp', '.torch')
if not exists(cache_dir):
    print("creating {}".format(cache_dir))
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    print("creating {}".format(models_dir))
    makedirs(models_dir)   

In [ ]:
# !cp ../input/resnet101/resnet101.pth /tmp/.torch/models/resnet101-5d3b4d8f.pth
# we need to find the weights of the pretrained model in https://www.kaggle.com/pytorch and add the weiths to our kernel.
# since pytorch finds the weights in /tmp/.torch/models, we need to copy the weights file to the folder and rename it.
!cp ../input/resnet50/resnet50.pth /tmp/.torch/models/resnet50-19c8e357.pth

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision as tv
from torchvision import transforms
# define transforms
img_size = (224, 224)
train_transforms = transforms.Compose([transforms.Resize(img_size), transforms.RandomHorizontalFlip(0.5), 
                                   transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(img_size), 
                                   transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
# define data loader
batch_size = 64
train_dataset = tv.datasets.ImageFolder("train1", train_transforms)
valid_dataset = tv.datasets.ImageFolder("val1", valid_transforms)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# define model
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
myModel = tv.models.resnet50(pretrained=True)
set_parameter_requires_grad(myModel, True)
num_ftrs = myModel.fc.in_features
num_classes = len(category_files.keys())
# three layers are added to the top of the pretrained model
myModel.fc = nn.Sequential(*[nn.Linear(num_ftrs, 1024), nn.Dropout(0.25), nn.Linear(1024, num_classes)])

# define optimizer and loss
optimizer = optim.SGD(myModel.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
epochs = 200
best_val_loss = 100
for i in range(epochs):
    train_loss = 0
    train_corrects = 0
    myModel.train()
    for inputs, labels in train_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        predicts = myModel(inputs)
        loss = criterion(predicts, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(predicts, 1)
        train_loss += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)
    val_loss = 0
    val_corrects = 0
    myModel.eval()
    for inputs, labels in valid_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        predicts = myModel(inputs)
        loss = criterion(predicts, labels)
        _, preds = torch.max(predicts, 1)
        val_loss += loss.item() * inputs.size(0)
        val_corrects += torch.sum(preds == labels.data)        
    print("epoch: {}, train loss: {}, train accu: {}, val loss: {}, val accu: {}".format(i, 
        train_loss / len(train_data_loader.dataset), train_corrects.double() / len(train_data_loader.dataset), 
        val_loss / len(valid_data_loader.dataset), val_corrects.double() / len(valid_data_loader.dataset)))
    if val_loss / len(valid_data_loader.dataset) < best_val_loss:
        torch.save(myModel.state_dict(), "myModel.pkl")
        best_val_loss = val_loss / len(valid_data_loader.dataset)